# Direct Fidelity Estimation of Quantum States

*Copyright (c) 2022 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## Outline

Quantum State Tomography gives complete information about the state we prepared, but it is very resource-consuming, and sometimes we only care about the fidelity between the ideal and actual quantum states. Direct Fidelity Estimation (DFE) is a simple method for certifying that how close between the actually prepared state and the ideal state.

This tutorial introduces Direct Fidelity Estimation (DFE) of quantum states, covering its theory and implementation on [Baidu Quantum Platform](https://quantum.baidu.com/).

## Theory


### Fidelity estimation

The fidelity between our desired pure states $\rho$ and the actual state $\sigma$ is given by

$$
    \mathcal{F}(\rho, \sigma) = \textrm{Tr}[\rho\sigma].
$$

We define $P_k(k=1,\cdots,d^2)$ as all possible Pauli operators, where $d=2^n$ and $n$ is the number of qubits. We know that both $\sigma$ and $\rho$ can be expanded in Pauli basis. Define the characteristic function $\chi_\rho(k) = \textrm{Tr}[\rho P_k/\sqrt{d}]$, and the fidelity $\mathcal{F}(\rho, \sigma)$ can be rewritten as

$$
\begin{align*}
        \textrm{Tr}[\rho\sigma]
    &=  \langle\langle \rho | \sigma \rangle\rangle \\
    &=  \sum_k \langle\langle \rho | k \rangle\rangle \langle\langle k | \sigma \rangle\rangle \\
    &=  \sum_k \textrm{Tr}[\rho P_k/\sqrt{d}]\textrm{Tr}[\sigma P_k/\sqrt{d}] \\
    &=  \sum_k \chi_\rho(k)\chi_\sigma(k).
\end{align*}
$$

Then we construct an estimator for $\textrm{Tr}[\rho\sigma]$ as follows. Firstly, we sample $k\in\left\{1, \cdots, d^2\right\}$ at random with probability

$$
    \textrm{Pr}(k) = \chi^2_\rho(k).
$$

For each $k$, we can estimate $\chi_\sigma(k)$ with $P_k$ measurement circuit. The estimator $X$ is defined as

$$
    X = \chi_\sigma(k) / \chi_\rho(k).
$$

We can easily find that $\mathbb{E}X = \sum_k\textrm{Pr}(k)X = \textrm{Tr}[\rho\sigma]$.

### Parameter setting

There remains two questions:

1. How much Pauli operators we have to sample?
2. For each Pauli operator, how to select the number of shots?

In the following, we will solve above question. Now we consider some fixed additive error $\epsilon$ and failure probability $\delta$. By Chebyshev's inequality and Hoeffding's inequality, we can compute the sample times $\ell$ and $i$-th Pauli operator's shot number $m_i$ with these two parameters[1].

We can conclude that, with probability $\geq 1-2\delta$, the fidelity $\mathcal{\rho,\sigma}$ lies in the range $[\tilde{\mathcal{F}}(\rho, \sigma)-2\epsilon, \tilde{\mathcal{F}}(\rho, \sigma)+2\epsilon]$ , where $\tilde{\mathcal{F}}(\rho, \sigma)$ is our estimation of fidelity.


## Practice

Then, we demonstrate DFE on the Bell state.

First, we import the necessary libraries.

In [ ]:
import QCompute
import Extensions.QuantumErrorProcessing.qcompute_qep.estimation as estimation

Then we set up the quantum program for preparing the Bell state on the second and third qubit.

In [ ]:
qp = QCompute.QEnv()  # qp is short for "quantum program", instance of QProgram
qp.Q.createList(3)
QCompute.H(qp.Q[1])
QCompute.CX(qp.Q[1], qp.Q[2])

![DFE](./figures/dfe-bell-state.png "Figure 1: We take the Bell state as the target of DFE. ")

The circuit is shown in the figure above.

Then we set the quantum computer (instance of QComputer). The QuantumComputer can be a simulator or a hardware interface. The rest is simple. We initialize a DFEState instance and call the estimate procedure. Here we set $\epsilon=\delta=0.05$ and choose the 2st and 3nd qubits.

In [ ]:
# For numeric test, use the local ideal simulator
qc = QCompute.BackendName.LocalBaiduSim2

# Please log in the "Quantum Leaf" platform (https://quantum-hub.baidu.com/) to get Token
# QCompute.Define.hubToken = "Token"
# qc = QCompute.BackendName.CloudBaiduQPUQian

# Initialize a DFEState instance
est = estimation.DFEState()

# Call the estimate procedure
est.estimate(qp, qc, qubits=[1, 2], epsilon=0.05, delta=0.05)

Finally, we can obtain the fidelity between ideal and actual Bell state.

In [ ]:
print("Fidelity between the ideal and noisy states is: F = {:.5f}".format(est.fidelity))

## References

[1] Flammia, Steven T., and Yi-Kai Liu. "Direct fidelity estimation from few Pauli measurements." **[Physical Review Letters](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.106.230501)** 106.23 (2011): 230501.

[2] da Silva, Marcus P., Olivier Landon-Cardinal, and David Poulin. "Practical characterization of quantum devices without tomography." **[Physical Review Letters](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.107.210404)** 107.21 (2011): 210404.

[3] Kliesch, Martin, and Ingo Roth. "Theory of Quantum System Certification." **[PRX Quantum](https://journals.aps.org/prxquantum/abstract/10.1103/PRXQuantum.2.010201)** 2.1 (2021): 010201.